## Import libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

## Read files

In [2]:
file_name = 'faces_data.csv'
column_names=['emotion','pixels','usage']
df=pd.read_csv(file_name,names=column_names, na_filter=False)
df

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
...,...,...,...
35883,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35884,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35885,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35886,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


## Read and split columns

In [3]:
def readlines(file_name):
    features = []
    target_variable = []
    first_column = True

    with open(file_name) as file_:
        for line in file_:
            if first_column:
                first_column = False
            else:
                row = line.split(',')
                target_variable.append(int(row[0]))
                features.append([int(p) for p in row[1].split()])
        features, target_variable = np.array(features) / 255.0, np.array(target_variable)
    return features, target_variable

## Reshape columns

In [4]:
features, target_variable = readlines(file_name)
num_class = len(set(target_variable))
print("In this dataset, we have ",num_class, "classes.")
#
features = features.reshape(features.shape[0], 48, 48, 1)
features.shape
#
print(features.shape)
print(features.size)
print(len(features))

In this dataset, we have  7 classes.
(35887, 48, 48, 1)
82683648
35887


## Split dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, target_variable, train_size=0.8, random_state=42, shuffle=False)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)
print("Train dataset shape",y_train.shape,"\n","Test dataset shape",y_test.shape)

Train dataset shape (28709, 7) 
 Test dataset shape (7178, 7)


## Apply the model

In [6]:
# Initialising CNN
model = Sequential()
# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# Flattening
model.add(Flatten())
# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

## Fit the model

In [7]:
path_model='model_output.h5'
# fit the model
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=10, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

Epoch 1/10
449/449 [==============================] - 945s 2s/step - loss: 2.0194 - accuracy: 0.2262 - val_loss: 1.9236 - val_accuracy: 0.2692
Epoch 2/10
449/449 [==============================] - 948s 2s/step - loss: 1.8374 - accuracy: 0.2850 - val_loss: 1.7694 - val_accuracy: 0.3128
Epoch 3/10
449/449 [==============================] - 923s 2s/step - loss: 1.7323 - accuracy: 0.3308 - val_loss: 1.6822 - val_accuracy: 0.3553
Epoch 4/10
449/449 [==============================] - 898s 2s/step - loss: 1.6390 - accuracy: 0.3688 - val_loss: 1.7111 - val_accuracy: 0.3657
Epoch 5/10
449/449 [==============================] - 882s 2s/step - loss: 1.5840 - accuracy: 0.3876 - val_loss: 1.6261 - val_accuracy: 0.3937
Epoch 6/10
449/449 [==============================] - 1196s 3s/step - loss: 1.5202 - accuracy: 0.4159 - val_loss: 1.8263 - val_accuracy: 0.3617
Epoch 7/10
449/449 [==============================] - 899s 2s/step - loss: 1.4598 - accuracy: 0.4365 - val_loss: 1.5574 - val_accuracy: 0.422

In [8]:
pred_y=model.predict(X_test)
print(pred_y)
print(y_test.shape)

[[0.19986106 0.0181735  0.43980545 ... 0.18841545 0.05887    0.08307365]
 [0.09861285 0.03665795 0.18158692 ... 0.15000238 0.03675894 0.18423952]
 [0.197835   0.0324081  0.20266789 ... 0.256772   0.01153912 0.29464996]
 ...
 [0.12121066 0.00521638 0.07529508 ... 0.28911346 0.0099457  0.29014105]
 [0.22287619 0.01299356 0.05565302 ... 0.32807148 0.00733717 0.20502801]
 [0.16569526 0.0170505  0.21747993 ... 0.46748635 0.00717183 0.10558068]]
(7178, 7)


## Save the model to json file to be used later on

In [9]:
model_json = model.to_json()
model.save_weights('model_weights.h5')
with open("model.json", "w") as json_file:
    json_file.write(model_json)